In [82]:
import os
from video_downloader import *
from os import environ

In [113]:
email = "fpolacov@gmail.com"
password = "aaaaaa"

semantics = ["lying", "sitting", "standing", "people", "background"]
bucket_groups_names = ["bucket_"+sem for sem in semantics]

# Paths
group_videos_path=environ.get("group_videos_path",'/media/Gui2/thermix/ARThermal/olmos_001_frames_no_movement/14_tim/3')
cnn_results=environ.get("cnn_results","/media/Gui2/thermix/ARTraining/trained_models/thermix_33a/olmos_001_poses_classify.txt")

In [114]:
from os import listdir
from os.path import join
from datetime import *
from operator import attrgetter
import requests

''' returns list of video folders from group ordered by time '''
def videos_ordered_by_time_for_group(group_path):
    files_and_dates = []
    
    files = listdir(group_path)
    for path in files:
        files_and_dates.append(get_file_and_date(path))
    
    files_and_dates.sort(key=lambda item:item['date'])
    
    sorted_files = [os.path.join(group_path,f['path']) for f in files_and_dates]
    return sorted_files


def order_video_list_by_time(video_list):
    # video_list = [video, semantic]+
    files_and_dates = []
    for path,semantic in video_list:
        files_and_dates.append([get_file_and_date(path), semantic])
    files_and_dates.sort(key=lambda item:item[0]['date'])
#    sorted_files = [os.path.join(group_path,f['path']) for f in files_and_dates]
    return [[f[0]['path'],f[1]] for f in files_and_dates]
    
''' returns time for a filename the way videos where downloaded '''
def get_file_and_date(filename):
    # split and remove .000*_+.mov
    split1 = filename.split(".")
    f1 = split1[0]
    
    # hardcode to split
    split2 = f1.split("unkown_")
    f2 = split2[1]
    
    # get date
    split3 = f2.split("_")
    date = split3[0]

    # get time
    time = split3[1]
    split4 = time.split("%3A")
    h = split4[0]
    m = split4[1]
    s = split4[2]
    
    # since time is in backend's timezone, hardcode it to fit Argentina's timezone
    # there's a problem when h > 01:00 h < 03:59 day will not change.
    h_real = (int(h) - 3) % 24
    
    d = datetime.strptime(date+" "+str(h_real)+":"+m+":"+s, "%Y-%m-%d %H:%M:%S")
    return { 'path':filename, 'date':d }

In [115]:
from os import listdir
from os.path import isfile, join, dirname

def get_semantics_for_videos(videos, cnn_results):

    cnn_files = open(cnn_results,'r')
    cnn_lines = cnn_files.readlines()
    cnn_files.close()

    return_dict = {}

    for result in cnn_lines:
        path, res, _  = result.split(' ')
        video_path_from_path = os.path.basename(os.path.dirname(path))
        
        if not video_path_from_path in return_dict:
            return_dict[video_path_from_path] = [res]
        else:
            return_dict[video_path_from_path].extend([res])

    return return_dict
    
def get_frames_from_video_path(video_path):
    files = [os.path.join(video_path,f) for f in listdir(video_path) if isfile(os.path.join(video_path, f))]
    return sorted(files)

In [116]:
''' returns list of videos that have all same semantics ordered by time '''
def videos_with_same_semantic_in_all_frames(videos):
    return_list = []
    
    #videos = {'video_path':[semantics]}
    for video, semantics in videos.iteritems():
        if len(set(semantics)) == 1:
            return_list.append([video,semantics[0]])
    
    return return_list

''' returns list of videos that have all semantics with semantic_id ordered by time '''
def videos_with_semantic(videos, semantic_id):
    return_list = []
    
    #videos = {'video_path':[semantics]}
    for video, semantics in videos.iteritems():
        if len(set(semantics)) == 1 and semantic_id in semantics:
            return_list.append([video,semantic_id])
    
    return return_list

''' returns list of videos that have at least a frame with semantic_id ordered by time '''
def videos_that_include_semantic(videos, semantic_id):
    return_list = []
    
    #videos = {'video_path':[semantics]}
    for video, semantics in videos.iteritems():
        if semantic_id in semantics:
            return_list.append([video,semantic_id])
    
    return return_list

''' returns all videos that changed their semantic from the previous one '''
def videos_that_changed_semantic_from_previous(videos):
    # videos = [path,semantic]+
    return_list = [videos[0]]
    for vid in videos[1:]:
        last_elem = return_list[-1]
        
        if not last_elem[1] == vid[1]:
            return_list.append(vid)
    
    return return_list

''' forwards videos to their corresponding bucket in backend '''
def forward_videos_in_bramble(videos):
    # user token for accessing API
    token = login(email, password)
    
    # [(group_name, chat_id),...]
    groups = get_groups_of_user_with_token(token)
    
    # {group_name: chat_id, } for all groups in bucket list
    bucket_groups = dict((gname,gid) for gname,gid in groups if gname in bucket_groups_names)
    
    # url for forwarding video
    forward_url = "https://dev.thermal.us.bramblexpress.com/api/v1/entries/verzus/forward_with_name/"
    i = 0
    for video,sem in videos:
        # index of semantic in semantic list
        sem_index = int(sem)-1
        
        # bucket group name for semantic index
        gname = bucket_groups_names[sem_index]

        # chat id for group
        chat_id = bucket_groups[gname]
        
        # forward video to bucket group of the corresponding semantic
        response = requests.post(forward_url,
                                 json={'chats':[int(chat_id)],'video_name':video+".mov"},
                                 verify=False,
                                 headers={'Authorization':'Token %s' % token})
        i +=1
    return i

In [117]:
# 
vids = [os.path.join(group_videos_path,f) for f in listdir(group_videos_path)]
videos_and_semantics = get_semantics_for_videos(vids, cnn_results)

# get videos with same semantics
#usable_videos = videos_with_same_semantic_in_all_frames(videos_and_semantics)
usable_videos = videos_with_semantic(videos_and_semantics, "2")

# order usable videos by date
ordered_videos_by_time = order_video_list_by_time(usable_videos)

# get videos with 'jumps' in semantics
#forward_list = videos_that_changed_semantic_from_previous(ordered_videos_by_time)
forward_list = ordered_videos_by_time

ok = forward_videos_in_bramble(forward_list)
if ok is not None:
    print "OK: %d"%ok
else:
    print "ERROR in forwards"

OK: 51
